<a href="https://colab.research.google.com/github/Sheryar-bit/Lang-Chain/blob/main/LangChain_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate bitsandbytes langchain langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.7 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

In [5]:
model_id = "openai-community/gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# A simple txt generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7,
)

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu


In [7]:
print("Q: Explain quantum computing in simple terms")
print("A:", llm.invoke("Explain quantum computing in simple terms"))

Q: Explain quantum computing in simple terms


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A: Explain quantum computing in simple terms, but have very little impact on the physical and theoretical fields of computing.

The following is from the paper by Mihail Shah, the first author of IEEE Spectrum (2011).

Quantum Computer Concepts

Quantum computing requires only the use of an ordinary method of computing any given state at all. A quantum computer is able to perform a real-world operation without losing information in the form of information. A real-world operation can be performed via a simple but powerful set


In [8]:
#Creating and testing a custom prompt template (apna banaya hua)
template = """You are an expert AI assistant. Provide detailed answers.
Question: {question}
Answer:"""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm #First applying the prompt template then will pass the result to the LLM(this is the modern way, traditional as parameter based)

response = chain.invoke({"question": "How to scale your backend server upto 1 million users?"})
print("\nDetailed ans:")
print(response )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Detailed ans:
You are an expert AI assistant. Provide detailed answers.
Question: How to scale your backend server upto 1 million users?
Answer: A simple answer: The easiest way to scale your backend server to 500 million users is to scale the backend server to 1500 users. In order to perform this task, you will need an initial amount of data.
Question: How to scale your backend client?
Answer: You can add a custom backend server to increase the size of your backend server.
Question: How to scale your database?
Answer: You can add a custom backend server to increase the size of your backend server.



In [10]:
# another advanced usage could be like multi conversations
conversation = [
    {"role": "user", "content": "Tell me about the history of the Pakistan"},
    {"role": "assistant", "content": "Pakistan was..."},
    {"role": "user", "content": "What are Pakistan cricket team greatest achievemnts?"}
]
#formatting a little bit
formatted_conv = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation])

#this will invoke ourr LLm
print(llm.invoke(f"Continue this conversation:\n{formatted_conv}\nassistant:"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Continue this conversation:
user: Tell me about the history of the Pakistan
assistant: Pakistan was...
user: What are Pakistan cricket team greatest achievemnts?
assistant: Pakistan have been in the top five in Test cricket
user: How did you reach the top two in your career
assistant: Pakistan has played in the top four in your career
user: Can you tell me about the Pakistan team captain? He became an assistant coach of the Pakistan team
user: What is your favourite part of playing cricket with the Pakistan team?
assistant: Pakistan has managed to play 100% of the matches
user: What is your favorite part of playing cricket
